# **SDXL Model notebook**

In [ ]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth):
    b = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(b)

print("Connecting...")
drive.mount('/content/gdrive')
mainpth = "MyDrive"
clear_output()
inf('✔ Done', 'success', '50px')

#@markdown ---

In [ ]:
#@markdown # Install/Update AUTOMATIC1111 Forge
from IPython.utils import capture
from IPython.display import clear_output, display
import ipywidgets as widgets
import os
import time

def inf(msg, style, wdth):
    b = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(b)

base_dir = f"/content/gdrive/{mainpth}/sd"
webui_dir = f"{base_dir}/stable-diffusion-webui"

with capture.capture_output():
    os.makedirs(base_dir, exist_ok=True)
    %cd {base_dir}

    if not os.path.exists(webui_dir):
        !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge stable-diffusion-webui
    else:
        %cd {webui_dir}
        !git pull

    # Cache dirs
    os.makedirs(f"{webui_dir}/cache", exist_ok=True)
    os.environ['TRANSFORMERS_CACHE'] = f"{webui_dir}/cache"
    os.environ['TORCH_HOME'] = f"{webui_dir}/cache"
    os.environ['HF_HOME'] = f"{webui_dir}/cache"

clear_output()
inf("✔ Done", "success", "80px")

#@markdown ---

In [ ]:
#@markdown # Install Python 3.11 and create venv (fix for Python 3.12 errors)
from IPython.display import clear_output
import os

print("Installing Python 3.11...")
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null 2>&1
!sudo apt update -q > /dev/null 2>&1
!sudo apt install python3.11 python3.11-venv python3.11-distutils -y -q > /dev/null 2>&1
!curl -s https://bootstrap.pypa.io/get-pip.py | python3.11 > /dev/null 2>&1

print("Creating venv with Python 3.11...")
%cd {webui_dir}
!python3.11 -m venv venv --system-site-packages
venv_python = "venv/bin/python"
venv_pip = "venv/bin/pip"

print("Installing dependencies in venv...")
!{venv_pip} install --upgrade pip setuptools wheel -q
!{venv_pip} install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!{venv_pip} install xformers -q
!{venv_pip} install -r requirements.txt -q
!{venv_pip} install -r requirements_versions.txt -q
!{venv_pip} install -U diffusers accelerate controlnet_aux jedi -q --no-deps

clear_output()
print("✔ Python 3.11 venv ready")


In [ ]:
#@markdown # Requirements (accelerate config)
print("Configuring accelerate...")
!{venv_python} -m accelerate config default || echo "Accelerate уже настроен"

# Очистка кэша
!rm -rf /root/.cache/huggingface/* /root/.cache/torch/*

print("Done. Проверь GPU:")
!nvidia-smi
!{venv_python} -c "import torch; print('CUDA available:', torch.cuda.is_available()); print('Device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')"


In [ ]:
import os
from IPython.display import clear_output, display
import ipywidgets as widgets

def inf(msg, style, wdth):
    b = widgets.Button(description=msg, disabled=True, button_style=style,
                       layout=widgets.Layout(min_width=wdth))
    display(b)

#@markdown ### Path to your model (on Google Drive)
MODEL_PATH = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/WAI-illustrious-SDXL_16.safetensors"  #@param {type:"string"}

WEBUI_DIR = f"/content/gdrive/{mainpth}/sd/stable-diffusion-webui"
MODELS_DIR = f"{WEBUI_DIR}/models/Stable-diffusion"

os.makedirs(MODELS_DIR, exist_ok=True)

if MODEL_PATH.strip() == "":
    inf("✘ Please provide MODEL_PATH", "danger", "300px")
elif not os.path.exists(MODEL_PATH):
    inf("✘ Model file not found! Check path.", "danger", "350px")
else:
    dst = os.path.join(MODELS_DIR, os.path.basename(MODEL_PATH))

    if not os.path.exists(dst):
        try:
            os.symlink(MODEL_PATH, dst)
        except:
            !cp "$MODEL_PATH" "$dst"

    clear_output()
    inf("✔ Model linked and ready", "success", "300px")
    print("Using model:", dst)

In [ ]:
#@markdown # ControlNet для SDXL (только SDXL-совместимые модели)
from torch.hub import download_url_to_file
from urllib.parse import urlparse
import os
from IPython.display import clear_output

# Путь для моделей ControlNet
CONTROLNET_DIR = f"{webui_dir}/models/ControlNet"
os.makedirs(CONTROLNET_DIR, exist_ok=True)

#@markdown Выберите, какие модели ControlNet скачать:
XL_ControlNet = "Union (рекомендую)" #@param ["None", "Union (рекомендую)", "Union Promax", "Canny", "Depth", "OpenPose", "Scribble", "Tile / Upscale", "All (несколько файлов)"]

# Актуальные ссылки на SDXL-модели (2025–2026)
MODELS = {
    "Union (рекомендую)": {
        "name": "controlnet-union-sdxl-1.0.safetensors",
        "url": "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors"
    },
    "Union Promax": {
        "name": "controlnet-union-sdxl-1.0-promax.safetensors",
        "url": "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/controlnet-union-sdxl-1.0-promax.safetensors"  # актуальную ссылку уточняйте
    },
    "Canny": {
        "name": "diffusers_xl_canny_full.safetensors",
        "url": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors"
    },
    "Depth": {
        "name": "diffusers_xl_depth_mid.safetensors",
        "url": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors"
    },
    "OpenPose": {
        "name": "thibaud_xl_openpose_256lora.safetensors",
        "url": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors"
    },
    "Scribble": {
        "name": "sai_xl_scribble_256lora.safetensors",
        "url": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_scribble_256lora.safetensors"
    },
    "Tile / Upscale": {
        "name": "diffusers_xl_tile.safetensors",
        "url": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_tile.safetensors"
    }
}

# Функция скачивания
def download_model(url, target_dir, filename=None):
    if not filename:
        filename = url.split("/")[-1].split("?")[0]
    dst = os.path.join(target_dir, filename)
    if os.path.exists(dst):
        print(f"✓ Уже существует: {filename}")
        return
    print(f"Скачиваю: {filename}")
    download_url_to_file(url, dst, progress=True)

# Установка/обновление расширения ControlNet
print("Установка / обновление ControlNet extension...")
with capture.capture_output():
    %cd {webui_dir}/extensions
    if not os.path.exists("sd-webui-controlnet"):
        !{venv_python} -m git clone https://github.com/Mikubill/sd-webui-controlnet.git
    else:
        %cd sd-webui-controlnet
        !{venv_python} -m git reset --hard
        !{venv_python} -m git pull
    %cd /content

# Скачивание выбранных моделей
print("\nСкачивание ControlNet моделей для SDXL...")

if XL_ControlNet == "None":
    print("Модели ControlNet не выбраны → пропускаем скачивание")
elif XL_ControlNet == "All (несколько файлов)":
    for key, data in MODELS.items():
        if key != "Union Promax":
            download_model(data["url"], CONTROLNET_DIR, data["name"])
else:
    if XL_ControlNet in MODELS:
        data = MODELS[XL_ControlNet]
        download_model(data["url"], CONTROLNET_DIR, data["name"])
    else:
        print("Выбранный вариант не найден в списке")

clear_output()
inf("✔ ControlNet для SDXL готов", "success", "300px")
print(f"Выбрано: {XL_ControlNet}")
print("Путь к моделям:", CONTROLNET_DIR)
print("После запуска webui зайдите в ControlNet → Refresh models")

In [ ]:
#@markdown # Запуск Stable Diffusion (Forge)

from IPython.utils import capture
import time
import sys

User = "nbvf"          #@param {type:"string"}
Password = "22334455"  #@param {type:"string"}

auth = f"--gradio-auth {User}:{Password}" if User and Password else ""

share = '--share'

print("Запуск Forge...")

!{webui_dir}/venv/bin/python {webui_dir}/webui.py {share} \
  --api \
  --disable-safe-unpickle \
  --enable-insecure-extension-access \
  --no-download-sd-model \
  --no-half-vae \
  --ckpt "{MODEL_PATH}" \
  --xformers \
  {auth} \
  --disable-console-progressbars \
  --skip-version-check \
  --opt-sdp-no-mem-attention \
  --always-offload-from-vram